In [ ]:
import org.apache.spark.sql.types.{StructType, StringType, IntegerType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [ ]:
println(s"Current spark version is ${spark.version}")

In [ ]:
val inputStreamPath = "/home/jovyan/work/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)
    .add("hiddentargetclue", IntegerType)
    .add("arrived_key", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

In [ ]:
val stream = inputDF.writeStream.foreachBatch {
    (batchDF: DataFrame, batchId: Long) => {
        print(batchDF.selectExpr("max(arrived_key)").take(1)(0)(0))
        print(13.toChar)
    }
}.start

In [ ]:
//val window = inputDF.writeStream.window(seconds(10), seconds(1)) {
    
//}

- https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier
- https://spark.apache.org/docs/2.3.3/sql-programming-guide.html
- https://spark.apache.org/docs/latest/streaming-programming-guide.html
- https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.streaming.DataStreamWriter

In [ ]:
stream.stop()